In [ ]:
import os
train_data_dir = '../keras/dataset/lfw_mtcnnpy_gray_picked_10_8_train_valid/train'
validation_data_dir = '../keras/dataset/lfw_mtcnnpy_gray_picked_10_8_train_valid/valid'
cnt = 0
cnt_ = 0
for subfolder in os.listdir(train_data_dir):
    cnt_ += 1
    subfolder_path = train_data_dir + "/" + subfolder
    for file_ in os.listdir(subfolder_path):
        cnt += 1
print "num of trainset is: ", cnt
print "num of trainset folder is: ", cnt_

cnt = 0
cnt_ = 0
for subfolder in os.listdir(validation_data_dir):
    cnt_ += 1
    subfolder_path = validation_data_dir + "/" + subfolder
    for file_ in os.listdir(subfolder_path):
        cnt += 1
print "num of valid is: ", cnt
print "num of trainset folder is: ", cnt_

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model 
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from utility import preprocess_image
from keras.models import load_model 
import sys
import os
sys.path.insert(0, 'model')
import tensorflow as tf

In [ ]:
#Get Training Config Parameters
run_dir = './train_run_picked_423_aug'
if not os.path.exists(run_dir):
    os.makedirs(run_dir)

nb_train_samples = 37002
nb_validation_samples = 4126
save_step = 10
epochs = 30000
batch_size = 64
learning_rate = 0.001
momentum = 0.99
fine_tuning_ture = 1
fine_tuning_model_path = "./" + run_dir + "/" + "weights-improvement-340-0.69-0.84-0.89.hdf5"
initial_epoch= 340

# GPU Usage Control
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [ ]:
#prepare network
# base model selection
bm = 6
# dimensions of our images.
img_width, img_height = 80,80
num_classes = 423
#0 -- VGG16 (224x224)
#1 -- Mobilenet (224x224)
#2 -- LeNet-5 (32x32)
#3 -- GesNet (32x32)
#4 -- ObjNet (80x80)
#5 -- ResNet-18 (224x224)
#6 -- FaceNet (80x80)
#7 -- VerifyNet (96x96)
#8 -- ResNet-50 (224x224)
#9 -- Yolo-v2-tiny (224x224)
#10-- Yolo-v2-tiny-lite (224x224)
#11-- Yolo-v2 (224x224)
#Strarting from random initialization
#if fine tuning from pre-trained model : weight = 'imagenet'
if(fine_tuning_ture == 0):
    #define input image size
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 1)
    
    #load predefined base network model
    if (bm==0):
        from keras.applications.vgg16 import VGG16 #VGG
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==1):
        from keras.applications.mobilenet import MobileNet #Mobilenet
        base_model = MobileNet(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==2):    
        from lenet import LeNet #LeNet-5
        base_model = LeNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==3):
        from gesnet import GesNet #GesNet
        base_model = GesNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==4):
        from objnet import ObjNet #ObjNet
        base_model = ObjNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==5):
        from resnet_18 import ResNet #ResNet-18
        base_model = ResNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==6):
        from facenet import FaceNet #FaceNet
        base_model = FaceNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==7):
        from verifynet_96_7x7 import VerifyNet #VerifyNet
        base_model = VerifyNet(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==8):
        from keras.applications.resnet50 import ResNet50 # ResNet-50
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==9):
        from yolo import Yolo_v2_tiny #Yolo_v2_tiny
        base_model = Yolo_v2_tiny(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==10):
        from yolo import Yolo_v2_tiny_lite #Yolo_v2_tiny_lite
        base_model = Yolo_v2_tiny_lite(include_top=False, weights=None, input_shape=input_shape)
    elif (bm==11):
        from yolo import Yolo_v2 #Yolo_v2
        base_model = Yolo_v2(include_top=False, weights=None, input_shape=input_shape)        
        
    #redefine last layer
    x = base_model.output
    #GesNet #ObjNet
    if(bm == 3 or bm == 4 or bm == 6 or bm == 7):
        x = Flatten()(x)
    #ResNet-18
    elif(bm == 5):
        x = AveragePooling2D(pool_size=(7, 7))(x)
        x = Flatten()(x)
    else:
        x = GlobalAveragePooling2D()(x)
    #convert to number of classes
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
else:
    model = load_model(fine_tuning_model_path)  

In [ ]:
model.summary()

In [ ]:
# prepare ImageDataGenerator
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    fill_mode='nearest',
    rotation_range=20,
    width_shift_range=0.9,
    height_shift_range=0.9,
    shear_range=0.2,
    zoom_range=0.2,
    preprocessing_function=preprocess_image)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True,
    )
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True)

In [ ]:
#start to train
#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate, momentum=momentum), metrics=['accuracy']    )

#setup check point
filepath=os.path.join(run_dir, "weights-improvement-{epoch:02d}-{val_loss:.2f}-{acc:.2f}-{val_acc:.2f}.hdf5")
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint(filepath,period=save_step)
callbacks_list = [checkpoint]

#start to train
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list,
    initial_epoch=initial_epoch)